In [4]:
import pandas as pd
from operator import itemgetter
import cPickle

dataset = 'Green'

attn_check_Qs = [("East", "West", "North"),
                 ("hot", "cold", "warm"),
                 ("king", "queen", "prince"),
                 ("sister", "brother", "aunt"),
                 ("hand", "fingers", "foot")]

all_data = pd.read_csv('Green1.0/trialdata_cleaned.csv', header = None, names = ['id', 'trial', 'time', 'data'])
data = all_data['data'].values

with open('AB_rel_dict.pickle', 'rb') as dictFile:
    AB_rel_dict = cPickle.load(dictFile)

answers_dict = {}
for trial in data:
    if trial[0] == '[':
        parts = trial.split('"')
        parts = [part.strip() for part in parts]
        
        # For SemEval, attach the corresponding relation to each question
        """
        if dataset == 'SemEval':
            AB = (parts[1], parts[3])
            relation = AB_rel_dict[AB]
            question = (relation, (parts[1], parts[3], parts[5]))
        else:
        """
        question = (parts[1], parts[3], parts[5])
        if question not in attn_check_Qs:
            answer = parts[7]
            if answer != 'SKIP':
                answer = answer.lower()
                if question in answers_dict:
                    ans_counts = answers_dict[question]
                    answers = [pair[0] for pair in ans_counts]
                    if answer in answers:
                        i = answers.index(answer)
                        ans_counts[i][1] += 1
                    else:
                        ans_counts.append([answer, 1])
                else:
                    answers_dict[question] = [[answer, 1]]

# Remove this bad question. I'm not sure why it's in there--maybe an intermediate version of the code?
#answers_dict.pop(('sister', 'brother', 'uncle'))

for answers in answers_dict.itervalues():
    answers.sort(key = itemgetter(1), reverse = True)
                
for question, answers in answers_dict.iteritems():
    print question
    print answers
    print

results_file = '../../word2vec-tests/data/analogy/analogy-completion/{}-results-no-attn-checks.pickle'.format(dataset)
with open(results_file, 'wb') as pickle_file:
    cPickle.dump(answers_dict, pickle_file, cPickle.HIGHEST_PROTOCOL)

('multiplication', 'product', 'addition')
[['sum', 21], ['subtraction', 5], ['product', 2], ['combination', 1], ['division', 1], ['adding', 1]]

('movie', 'screen', 'lightning')
[['sky', 17], ['storm', 4], ['bolt', 2], ['camera', 2], ['thunder', 2], ['dark', 1], ['thrunder', 1], ['lightning', 1], ['outlet', 1], ['lamp', 1]]

('pen', 'pig', 'reservoir')
[['water', 22], ['lake', 2], ['container', 2], ['fish', 2], ['hole', 1], ['dog', 1], ['duck', 1]]

('bracelet', 'wrist', 'moat')
[['castle', 26], ['arm', 1], ['moat', 1], ['fort', 1], ['king', 1], ['water', 1], ['boat', 1]]

('sister', 'brother', 'uncle')
[['aunt', 1]]

('jacket', 'zipper', 'wound')
[['stitches', 8], ['bandage', 6], ['bandaid', 3], ['blood', 2], ['cut', 2], ['stitch', 2], ['suture', 1], ['dehiscence', 1], ['gauze', 1], ['scab', 1], ['opening', 1], ['skin', 1], ['tear', 1]]

('burger', 'bun', 'book')
[['cover', 11], ['shelf', 7], ['pages', 3], ['page', 2], ['binding', 2], ['paper', 2], ['story', 1], ['bookend', 1], ['book

In [60]:
import cPickle, random

dataset = 'SemEval'
results_file = '../../word2vec-tests/data/analogy/analogy-completion/{}-results.pickle'.format(dataset)
with open(results_file, 'rb') as pickle_file:
    answers_dict = cPickle.load(pickle_file)
    
rand_index = random.randint(0, len(answers_dict))
question = answers_dict.keys()[rand_index]
answers = answers_dict[question]

print question
print answers

('10c', ('diary', 'person', 'signature'))
[['person', 6], ['name', 3], ['contract', 2], ['pen', 2], ['paper', 2], ['hand', 2], ['individual', 2], ['identity', 1], ['line', 1], ['guardian', 1], ['writing', 1], ['supervisor', 1], ['glimpse', 1], ['sign', 1], ['man', 1], ['owner', 1], ['page', 1], ['letter', 1]]


In [61]:
question = ('8b', ('lateness', 'hurry', 'tiredness'))
answers = answers_dict[question]
print answers

[['sleep', 11], ['sleepy', 6], ['rest', 2], ['awake', 2], ['sleepiness', 1], ['fatigue', 1], ['slowness', 1], ['energy', 1], ['cool', 1], ['sluggish', 1], ['pep', 1], ['inattention', 1], ['groggy', 1], ['coffee', 1]]


In [7]:
question = ('answer', 'riddle', 'solution')
answers = answers_dict[question]
print answers

[['problem', 22], ['question', 4], ['puzzle', 2], ['thinking', 1], ['plan', 1]]


In [10]:
question = ('answer', 'riddle', 'key')
answers = answers_dict[question]
print answers

import numpy as np
words, freqs = zip(*answers)

freqs = np.array(freqs)
props = freqs / float(np.sum(freqs))

print props

[['lock', 17], ['door', 4], ['puzzle', 2], ['unlock', 1], ['map', 1], ['item', 1], ['code', 1], ['keyhole', 1], ['riddle', 1], ['test', 1], ['question', 1]]
[ 0.5483871   0.12903226  0.06451613  0.03225806  0.03225806  0.03225806
  0.03225806  0.03225806  0.03225806  0.03225806  0.03225806]


In [32]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.plotly as py

init_notebook_mode(connected=True)

In [51]:
data = [go.Bar(x=words, y=props,
               marker=dict(color='rgb(100, 100, 100)'))]

layout = go.Layout(annotations=[dict(x=word,
                                     y=prop,
                                     text=str(freq),
                                     xanchor='center',
                                     yanchor='bottom',
                                     showarrow=False) for word, freq, prop in zip(words, freqs, props)],
                   title='Responses to <i>answer</i> : <i>riddle</i> :: <i>key</i> : ?',
                   font=dict(size = 14),
                   bargap = 0.3,
                   yaxis = dict(showgrid = False)
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)